In [1]:
# !pip install argparse
# !pip install beautifulsoup4
# !pip install spacy
# !python -m spacy download en_core_web_sm
# pip install pandarallel

# Begin

In [2]:
import logging
import os
import re
import spacy
en_nlp = spacy.load("en_core_web_sm")
import nltk
import numpy as np
import pathlib
import pandas as pd

from argparse import ArgumentParser
from bs4 import BeautifulSoup
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 36 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
def doc2tokens(doc):
    split_doc = doc.split('\n\n')
    headline, text = split_doc[0], split_doc[1]
    return pd.Series([headline, text])

with open('filter_word', 'r') as file:
    filter_list = file.read().split('\n')
filter_rule = '\w*|'.join(filter_list)
filter_rule =filter_rule[0:-4]
filter_rule

'threat\\w*|protest\\w*|strike\\w*|demonstra\\w*|violen\\w*|militar\\w*|coerc\\w*|repress\\w*|attack\\w*|bomb\\w*|terroris\\w*|assault\\w*|kill\\w*|tortur\\w*|curefew\\w*|destroy\\w*|explo\\w*|fight\\w*|artillery\\w*|weapon\\w*|mutilat\\w*|assassin'

# Now preprocess single chuck
we have totally 10 chucks. 

In [11]:
idx = 0
Num_Match =1
num_rows = 100000  # 0 - 873909


## Load a chuck

In [13]:
idx = 0
df = pd.read_csv(f'chuck_{idx}.csv',header=[0], nrows = num_rows)
df = df.iloc[:,1:]
df

,filename,docs
0,cna_eng_200307,JAPAN URGED TO PERMIT TOURISTS FROM TAIWAN TW...
1,cna_eng_200307,TAIWAN MUST FORGE NATIONAL WILL TO DEFEND ITS...
2,cna_eng_200307,RELEASE OF CHINA STEEL SHARES MAY ONLY BENEFI...
3,cna_eng_200307,DPP LAWMAKER TOUTS FOR TAICHUNG FREE TRADE PO...
4,cna_eng_200307,TAIWAN TO ORGANIZE ASIA-PACIFIC NGO FORUM IN ...
...,...,...
873905,xin_eng_200001,Australian Cyclist Banned for Two Years \n\n...
873906,xin_eng_200001,Australian Set World Record for Longest Run ...
873907,xin_eng_200001,Chinese Defense Minister Back from Four-Natio...
873908,xin_eng_200001,CPPCC Vice-Chairman Leaves for Davos \n\n Vi...


## extract headlines and texts from docs

In [6]:
df[['headlines','texts']]= df['docs'].parallel_apply(doc2tokens)
df = df.drop(columns=['docs'])
df

,filename,headlines,texts
0,cna_eng_200307,JAPAN URGED TO PERMIT TOURISTS FROM TAIWAN TW...,President Chen Shui-bian expressed the hope T...
1,cna_eng_200307,TAIWAN MUST FORGE NATIONAL WILL TO DEFEND ITS...,If mainland China were to attack Taiwan in a ...
2,cna_eng_200307,RELEASE OF CHINA STEEL SHARES MAY ONLY BENEFI...,Lawmakers alleged Thursday that the governmen...
3,cna_eng_200307,DPP LAWMAKER TOUTS FOR TAICHUNG FREE TRADE PO...,A lawmaker of the ruling Democratic Progressi...
4,cna_eng_200307,TAIWAN TO ORGANIZE ASIA-PACIFIC NGO FORUM IN ...,Taiwan will organize the country's first Asia...
...,...,...,...
99995,cna_eng_200501,ACID RAIN STILL PRESENT IN NORTHERN TAIWAN,Up to 70 percent of the rain that falls in no...
99996,cna_eng_200501,TAIWAN PAID US$36 PER BARREL OF OIL ON AVERAG...,Taiwan paid US$36 a barrel on average for imp...
99997,cna_eng_200501,PREMIER-DESIGNATE GATHERS NEW CABINET FOR PEP...,Premier-designate Frank Hsieh hosted an infor...
99998,cna_eng_200501,EPA HEAD MIFFED AT TAIWAN'S DISMAL RATING IN ...,Taiwan's Environmental Protection Administrat...


In [7]:
df[['match']] = df['texts'].parallel_apply(lambda x: re.findall(filter_rule,x))
df

,filename,headlines,texts,match
0,cna_eng_200307,JAPAN URGED TO PERMIT TOURISTS FROM TAIWAN TW...,President Chen Shui-bian expressed the hope T...,[]
1,cna_eng_200307,TAIWAN MUST FORGE NATIONAL WILL TO DEFEND ITS...,If mainland China were to attack Taiwan in a ...,"[attack, strike, military, attacks, military, ..."
2,cna_eng_200307,RELEASE OF CHINA STEEL SHARES MAY ONLY BENEFI...,Lawmakers alleged Thursday that the governmen...,[]
3,cna_eng_200307,DPP LAWMAKER TOUTS FOR TAICHUNG FREE TRADE PO...,A lawmaker of the ruling Democratic Progressi...,[]
4,cna_eng_200307,TAIWAN TO ORGANIZE ASIA-PACIFIC NGO FORUM IN ...,Taiwan will organize the country's first Asia...,[]
...,...,...,...,...
99995,cna_eng_200501,ACID RAIN STILL PRESENT IN NORTHERN TAIWAN,Up to 70 percent of the rain that falls in no...,[]
99996,cna_eng_200501,TAIWAN PAID US$36 PER BARREL OF OIL ON AVERAG...,Taiwan paid US$36 a barrel on average for imp...,[]
99997,cna_eng_200501,PREMIER-DESIGNATE GATHERS NEW CABINET FOR PEP...,Premier-designate Frank Hsieh hosted an infor...,[killful]
99998,cna_eng_200501,EPA HEAD MIFFED AT TAIWAN'S DISMAL RATING IN ...,Taiwan's Environmental Protection Administrat...,[]


In [8]:
# Num_Match =1

df = df[df.match.str.len() >= Num_Match]
df

,filename,headlines,texts,match
1,cna_eng_200307,TAIWAN MUST FORGE NATIONAL WILL TO DEFEND ITS...,If mainland China were to attack Taiwan in a ...,"[attack, strike, military, attacks, military, ..."
7,cna_eng_200307,SOUTHERN TAIWAN FREE FROM THREAT OF WATER SHO...,Although there is an increased threat of a po...,[threat]
18,cna_eng_200812,"Taichung landfill slide leaves 1 dead, 1 miss...",A series of landslides at a landfill retainin...,"[fighter, fighters, fighter, fighter, fighting]"
19,cna_eng_200812,Interior minister visits family of dead volun...,Minister of the Interior Liao Liou-yi on Sund...,"[fighter, killed, fighting, fighter, fighters,..."
22,cna_eng_200812,Body of buried worker at Taichung landfill re...,The body of a worker who was buried in dirt f...,[fighter]
...,...,...,...,...
99972,cna_eng_200501,TAIWAN SEVERS FORMAL DIPLOMATIC RELATIONS WIT...,The Republic of China on Taiwan severed forma...,[protest]
99986,cna_eng_200501,ACADEMIA SINICA PRODUCES DIGITAL TIME/SPACE E...,History readers have often felt something is ...,[military]
99987,cna_eng_200501,NEW NAVAL COMMANDER-IN-CHIEF NAMED,President Chen Shui-bian has appointed Chen P...,[military]
99989,cna_eng_200501,ACADEMIA SINICA PRESIDENT CALLS PUBLIC ATTENT...,Academia Sinica President Lee Yuan-tseh calle...,[killers]


In [9]:
df.match.str.len().value_counts().plot.bar(figsize=(12,8), rot=0)

<AxesSubplot:>

## examples to see why sport is confusing

In [10]:
sport_rule = 'football|soccer|basketball'

df[['sport']] = df['texts'].parallel_apply(lambda x: re.findall(sport_rule,x))
df[df.sport.str.len() >= Num_Match]

/home/yibo/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
/home/yibo/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/yibo/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on

,filename,headlines,texts,match,sport
3088,cna_eng_200608,U.S. BASKETBALL STAR TO VISIT TAIWAN,"U.S. basketball star Kobe Bryant, who plays f...",[demonstration],"[basketball, basketball]"
3759,cna_eng_201004,Taiwan operators display WiMAX applications a...,Six Taiwan operators showcased WiMAX applicat...,"[fighters, military]",[football]
3846,cna_eng_201004,Taiwanese basketball player shares 'go west' ...,A star Taiwanese professional basketball play...,"[kills, kills]","[basketball, basketball, basketball]"
3902,cna_eng_201004,Taiwan players dream of hitting the hardwood ...,"Despite limited success, most Taiwanese baske...","[kill, kills]","[basketball, basketball, basketball, basketbal..."
5083,cna_eng_200601,ESLITE PUTS TAIPEI ON THE MAP WITH MEGA-BOOKS...,Standing in the shadow of the world's talles...,[kills],[football]
...,...,...,...,...,...
91074,cna_eng_201007,First Taiwanese-American NBA player to return...,"Jeremy Lin, the first Taiwanese-American pl...",[kills],"[basketball, basketball]"
95623,cna_eng_200903,Paiwan woman fighting uphill battle against b...,"Vuvu Cjuku is on a mission. For three years, ...","[military, killing]",[football]
95643,cna_eng_200903,Paiwan woman fighting uphill battle against b...,"Vuvu Cjuku is on a mission. For three years, ...","[military, killing]",[football]
97412,cna_eng_200510,CHUNG CHENG ARMED FORCES PREPARATORY SCHOOL T...,Minister of National Defense Lee Jye told leg...,[military],[basketball]
